In [8]:
import os
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm
import tarfile 
import time

year = 2011
yearRef = 2006
listIPC = ["G06F", "G01N", "A61B", "B60L", "E21B", "F03D", "H01L", "H04W", "C07D", "D07B", "B32B"]
pathData = "/home/edgarlanoue/data/data" #"/home/edgarlanoue/data/data" # "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/data"
pathOutput = "/home/edgarlanoue/data/csv"  #"/home/edgarlanoue/data/csv" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données" 
batch_size= 1

pathYear = pathData + f"/{yearRef}/"  # Updates with variable year
jsonNamesYear = [f for f in listdir(pathYear) if isfile(join(pathYear, f))]

patent_number, titles, backgrounds, claims, summary, abstract, main_ipc, labels, sec_ipc, yearRefVec = {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}
patent_numberE, titlesE, backgroundsE, claimsE, summaryE, abstractE, main_ipcE, labelsE, sec_ipcE, yearRefVecE = {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}, {ipc: [] for ipc in listIPC}

current_date = int(f"{year}0101")

# Create a dictionary to store the expected classes for each IPC
expect_classes_ipc_dict = {}

# Initialize dictionaries to hold dataframes by IPC and yearRef
df_KS_dict = {ipc: {} for ipc in listIPC}
df_ES_dict = {ipc: {} for ipc in listIPC}

# Load expected classes for each IPC
for ipc in listIPC:
    expect_classes_ipc_yearRef = []
    with open(pathOutput + f'/ES/text/{year}_{ipc}_expectation_IPC_class.txt', 'r') as fp:
        for line in fp:
            x = line.strip()
            expect_classes_ipc_yearRef.append(x)  # Adjust based on your requirements
    expect_classes_ipc_dict[ipc] = expect_classes_ipc_yearRef

print(f"Iterating through patents of reference year {yearRef} for evalYear {year}")

total_files = len(jsonNamesYear)

# Creates lists for both Knowledge Space (KS) and Expectation Space (ES) with batch-size tqdm
with tqdm(total=total_files-200000, desc='Processing patents') as pbar:
    for i in range(200000, total_files, batch_size):
        for j in range(i, min(i + batch_size, total_files)):  # Process in batches
            patent_path = pathYear + jsonNamesYear[j]
            with open(patent_path) as f:
                d = json.load(f)  # Load JSON in d

            class_mainIPC = d['main_ipcr_label']
            class_main = class_mainIPC[0:4]

            # Collect all documents related to the main class for all IPCs
            for ipc in listIPC:
                # Create Knowledge Space (KS) for this IPC
                if class_main == ipc:
                    if int(d['date_published']) < current_date or d['decision'] in ['ACCEPTED', 'REJECTED']:
                        patent_number[ipc].append(d['application_number'])
                        titles[ipc].append(d['title'])
                        backgrounds[ipc].append(d['background'])
                        claims[ipc].append(d['claims'])
                        summary[ipc].append(d['summary'])
                        abstract[ipc].append(d['abstract'])
                        main_ipc[ipc].append(d['main_ipcr_label'])
                        labels[ipc].append(d['decision'])
                        sec_ipc[ipc].append(d['ipcr_labels'])
                        yearRefVec[ipc].append(yearRef)

                # Create Expectation Space (ES) for this IPC
                if class_mainIPC in expect_classes_ipc_dict[ipc]:
                    if int(d['date_published']) < current_date or d['decision'] in ['ACCEPTED', 'REJECTED']:
                        patent_numberE[ipc].append(d['application_number'])
                        titlesE[ipc].append(d['title'])
                        backgroundsE[ipc].append(d['background'])
                        claimsE[ipc].append(d['claims'])
                        summaryE[ipc].append(d['summary'])
                        abstractE[ipc].append(d['abstract'])
                        main_ipcE[ipc].append(d['main_ipcr_label'])
                        labelsE[ipc].append(d['decision'])
                        sec_ipcE[ipc].append(d['ipcr_labels'])
                        yearRefVecE[ipc].append(yearRef)

        # Update the progress bar after processing each batch
        pbar.update(min(batch_size, total_files - i))

for ipc in listIPC:
# Store KS dataframe in the nested dictionary
    df_KS_dict[ipc][yearRef] = pd.DataFrame({
        'application_number': patent_number[ipc],
        'title': titles[ipc],
        'abstract': abstract[ipc],
        'claims': claims[ipc],
        'background': backgrounds[ipc],
        'summary': summary[ipc],
        'ipc': main_ipc[ipc],
        'sec_ipc': sec_ipc[ipc],
        'label': labels[ipc],
        'yearRef': yearRefVec[ipc]
    })
    # Store ES dataframe in the nested dictionary
    df_ES_dict[ipc][yearRef] = pd.DataFrame({
        'application_number': patent_numberE[ipc],
        'title': titlesE[ipc],
        'abstract': abstractE[ipc],
        'claims': claimsE[ipc],
        'background': backgroundsE[ipc],
        'summary': summaryE[ipc],
        'ipc': main_ipcE[ipc],
        'sec_ipc': sec_ipcE[ipc],
        'label': labelsE[ipc],
        'yearRef': yearRefVecE[ipc]
    })



Iterating through patents of reference year 2006 for evalYear 2011


Processing patents:  74%|███████▎  | 80448/109449 [22:38<08:09, 59.20it/s] 


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [13]:
patent_path = pathYear + jsonNamesYear[280448]
with open(patent_path) as f:
    content = f.read()  # Read the file content first
    print(content)  # Check if the content is valid JSON    
    d = json.load(f)  # Load JSON in d

JSONDecodeError: Expecting value: line 1 column 1 (char 0)